In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import ttk, messagebox
from datetime import datetime

# Backtesting Strategies
def moving_average_crossover(data, short_window=50, long_window=200):
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['Close']
    signals['short_mavg'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
    signals['long_mavg'] = data['Close'].rolling(window=long_window, min_periods=1).mean()
    signals['signal'] = 0.0
    
    # Use .loc for label-based indexing
    signals.loc[short_window:, 'signal'] = np.where(
        signals['short_mavg'].iloc[short_window:] > signals['long_mavg'].iloc[short_window:], 1.0, 0.0
    )
    
    signals['positions'] = signals['signal'].diff()
    return signals

def mean_reversion(data, window=20):
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['Close']
    signals['mean'] = data['Close'].rolling(window=window).mean()
    signals['std'] = data['Close'].rolling(window=window).std()
    signals['signal'] = np.where(data['Close'] < (signals['mean'] - 2 * signals['std']), 1.0, 0.0)
    signals['positions'] = signals['signal'].diff()
    return signals

def momentum(data, window=14):
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['Close']
    signals['momentum'] = data['Close'].diff(window)
    signals['signal'] = np.where(signals['momentum'] > 0, 1.0, 0.0)
    signals['positions'] = signals['signal'].diff()
    return signals

def bollinger_bands(data, window=20):
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['Close']
    signals['mean'] = data['Close'].rolling(window=window).mean()
    signals['std'] = data['Close'].rolling(window=window).std()
    signals['upper_band'] = signals['mean'] + (signals['std'] * 2)
    signals['lower_band'] = signals['mean'] - (signals['std'] * 2)
    signals['signal'] = np.where(data['Close'] < signals['lower_band'], 1.0, 0.0)
    signals['positions'] = signals['signal'].diff()
    return signals

def rsi_strategy(data, window=14):
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['Close']
    signals['rsi'] = rsi
    signals['signal'] = np.where(rsi < 30, 1.0, np.where(rsi > 70, -1.0, 0.0))
    signals['positions'] = signals['signal'].diff()
    return signals

# Calculate Returns
def calculate_returns(data, signals):
    data['returns'] = data['Close'].pct_change()
    data['strategy_returns'] = signals['positions'].shift(1) * data['returns']
    data['cumulative_returns'] = (1 + data['strategy_returns']).cumprod()
    return data

# Generate HTML Report
def generate_html_report(data, strategy_name):
    html = f"""
    <html>
    <head>
        <title>{strategy_name} Backtest Results</title>
        <style>
            body {{ background-color: black; color: white; font-family: Arial, sans-serif; }}
            h1 {{ color: cyan; }}
            table {{ width: 100%; border-collapse: collapse; }}
            th, td {{ padding: 10px; text-align: left; border-bottom: 1px solid #444; }}
            th {{ background-color: #333; }}
            tr:hover {{ background-color: #222; }}
        </style>
    </head>
    <body>
        <h1>{strategy_name} Backtest Results</h1>
        <table>
            <tr><th>Date</th><th>Price</th><th>Signal</th><th>Positions</th><th>Cumulative Returns</th></tr>
    """
    for index, row in data.iterrows():
        html += f"""
            <tr>
                <td>{index.strftime('%Y-%m-%d')}</td>
                <td>{row['price']:.2f}</td>
                <td>{row['signal']}</td>
                <td>{row['positions']}</td>
                <td>{row['cumulative_returns']:.2f}</td>
            </tr>
        """
    html += """
        </table>
    </body>
    </html>
    """
    with open(f"{strategy_name}_backtest_results.html", "w") as file:
        file.write(html)

# GUI Application
class BacktestApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Backtesting Tool")
        self.root.configure(bg="black")
        
        # Ticker Input
        self.ticker_label = tk.Label(root, text="Enter Ticker:", fg="white", bg="black")
        self.ticker_label.grid(row=0, column=0, padx=10, pady=10)
        self.ticker_entry = tk.Entry(root, bg="gray", fg="white")
        self.ticker_entry.grid(row=0, column=1, padx=10, pady=10)
        
        # Strategy Selection
        self.strategy_label = tk.Label(root, text="Select Strategy:", fg="white", bg="black")
        self.strategy_label.grid(row=1, column=0, padx=10, pady=10)
        self.strategy_var = tk.StringVar()
        self.strategy_combobox = ttk.Combobox(root, textvariable=self.strategy_var, state="readonly")
        self.strategy_combobox['values'] = ("Moving Average Crossover", "Mean Reversion", "Momentum", "Bollinger Bands", "RSI Strategy")
        self.strategy_combobox.grid(row=1, column=1, padx=10, pady=10)
        self.strategy_combobox.current(0)
        
        # Returns Button
        self.returns_button = tk.Button(root, text="Generate Returns", command=self.generate_returns, bg="gray", fg="white")
        self.returns_button.grid(row=2, column=0, columnspan=2, padx=10, pady=10)
    
    def generate_returns(self):
        ticker = self.ticker_entry.get()
        strategy = self.strategy_var.get()
        
        if not ticker:
            messagebox.showerror("Error", "Please enter a ticker.")
            return
        
        try:
            # Fetch Data
            data = yf.download(ticker, start="2020-01-01", end=datetime.now().strftime('%Y-%m-%d'))
            if data.empty:
                messagebox.showerror("Error", f"No data found for ticker: {ticker}")
                return
            
            # Apply Strategy
            if strategy == "Moving Average Crossover":
                signals = moving_average_crossover(data)
            elif strategy == "Mean Reversion":
                signals = mean_reversion(data)
            elif strategy == "Momentum":
                signals = momentum(data)
            elif strategy == "Bollinger Bands":
                signals = bollinger_bands(data)
            elif strategy == "RSI Strategy":
                signals = rsi_strategy(data)
            else:
                messagebox.showerror("Error", "Invalid strategy selected.")
                return
            
            # Calculate Returns
            results = calculate_returns(data, signals)
            
            # Generate HTML Report
            generate_html_report(results, strategy)
            messagebox.showinfo("Success", f"HTML report generated for {strategy}.")
        
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

# Run Application
if __name__ == "__main__":
    root = tk.Tk()
    app = BacktestApp(root)
    root.mainloop()

C:\Users\Wolfrank\AppData\Local\Temp\ipykernel_3052\3424736186.py:146: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start="2020-01-01", end=datetime.now().strftime('%Y-%m-%d'))
[*********************100%***********************]  1 of 1 completed
